# Interpolate w' to the same vertical grid as b'
$w$ is on the `depthw` vertical grid and $b$ is on the `deptht` grid but to compute $w'b'$ they need to be on the same grid, so we interpolate $w$ to the `deptht` grid.

In [ ]:
import numpy as np
import xarray as xr
from glob import glob
import nc_time_axis
import cftime

In [ ]:
def prepro(ds):
    return ds.isel(y=slice(500, None))

Define functions to interpolate

In [ ]:
def interp_vertically(ds):
    interp = (ds + np.roll(ds,-1,axis=0))/2
    return interp

def interp_ufunc(data):
    interp = xr.apply_ufunc(interp_vertically, data, input_core_dims=[["time_counter","x","y"]], output_core_dims=[["time_counter","x","y"]], 
            dask="allowed", vectorize = True)
    return interp

Load grid files

In [ ]:
grid_files = "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mesh_hgr.nc"
coords_file = "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/coordinates_CREG12_lbclnk_noz_vh20160930.nc"
mask_file= "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mask.nc"

In [ ]:
grid = xr.open_mfdataset(grid_files, parallel=True, preprocess=prepro)
coords = xr.open_mfdataset(coords_file, parallel=True, preprocess=prepro)
mask = xr.open_mfdataset(mask_file, parallel=True, preprocess=prepro)

Load data files

In [ ]:
Bprime_data_filesREF = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-BUOY_prime/*2014*BUOYprime.nc"
Bprime_REF= xr.open_mfdataset(Bprime_data_filesREF, parallel=True, decode_times=False, concat_dim='time_counter', combine='nested')

In [ ]:
Wprime_data_filesREF = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-gridW_prime/*gridWprime.nc"
Wprime_data_filesFUT = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-gridW_prime/*gridWprime.nc"

Set coordinates and chunk the data

In [ ]:
Wprime_REF = xr.open_mfdataset(Wprime_data_filesREF, preprocess=prepro, parallel=True)
Wprime_REF = Wprime_REF.assign_coords({"nav_lon": grid.nav_lon, "nav_lat": grid.nav_lat})
Wprime_REF = Wprime_REF.chunk({"x": 395, "y": 501})
Wprime_FUT= xr.open_mfdataset(Wprime_data_filesFUT, preprocess=prepro, parallel=True)
Wprime_FUT = Wprime_FUT.assign_coords({"nav_lon": grid.nav_lon, "nav_lat": grid.nav_lat})
Wprime_FUT = Wprime_FUT.chunk({"x": 395, "y": 501})

Loop through time and do the interpolation at each time step, then save the interpolated data to disk.

In [ ]:
output = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-gridW_prime_interp/"

for time in Wprime_REF.time_counter:
    date = (str(time.values)[0:10])
    print(date)
    year, month, day = date.split("-")  
    data = Wprime_REF.sel(time_counter=date)
    interp_data = interp_ufunc(data)
    interp_data["depthw"] = grid.nav_lev.values
    interp_data.to_netcdf(output+"CREG12.L75-REF08_y{0}m{1}d{2}.5d_gridWprime.nc".format(year, month, day))

In [ ]:
output = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-gridW_prime_interp/"

for time in Wprime_FUT.time_counter:
    date = (str(time.values)[0:10])
    print(date)
    year, month, day = date.split("-")   
    data = Wprime_FUT.sel(time_counter=date)
    interp_data = interp_ufunc(data)
    interp_data["depthw"] = grid.nav_lev.values
    interp_data.to_netcdf(output+"CREG12.L75-FUT08_y{0}m{1}d{2}.5d_gridWprime.nc".format(year, month, day))